# Summarization

In [1]:
from transformers import pipeline
from tqdm.notebook import tqdm
import json, requests

In [0]:
TEXT_TO_SUMMARIZE = """ 
New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York. 
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband. 
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other. 
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage. 
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the 
2010 marriage license application, according to court documents. 
Prosecutors said the marriages were part of an immigration scam. 
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further. 
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective 
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002. 
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say. 
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages. 
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted. 
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s 
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali. 
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force. 
If convicted, Barrientos faces up to four years in prison. Her next court appearance is scheduled for May 18.
"""

In [0]:
def tasks(config_dict, search=None):
  try:
    print('Tasks:')
    for t in config_dict['task_specific_params'].keys():
      print('- ' + str(t))
    print('\n')
    
    if search in config_dict['task_specific_params'].keys():
      print(f'Detalles de {search}')
      print(config_dict['task_specific_params'][search])
    
    else:
      print('El modelo No está diseñado para esa tarea')
  
  except:
    return print('! No tiene tareas específicas')

## BART

In [0]:
# Use BART - PyTorch
'''
https://huggingface.co/facebook/bart-large-xsum
model = AutoModelWithLMHead.from_pretrained("facebook/bart-large-xsum")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-xsum")
'''

BART_summarizer = pipeline("summarization", model="bart-large-xsum", tokenizer="bart-large-xsum")

In [0]:
details = requests.get("https://s3.amazonaws.com/models.huggingface.co/bert/facebook/bart-large-xsum/config.json")
bart_large_config = json.loads(details.text)
tasks(bart_large_config, search='summarization')

Tasks:
- summarization


Detalles de summarization
{'early_stopping': True, 'length_penalty': 2.0, 'max_length': 142, 'min_length': 56, 'no_repeat_ngram_size': 3, 'num_beams': 4}


In [0]:
bart_summary = BART_summarizer(TEXT_TO_SUMMARIZE, min_length=56, max_length=250)
print(bart_summary)

[{'summary_text': 'A New York woman has pleaded not guilty to charges that she lied on a marriage license in order to get married 10 times, including to eight different men from different countries, prosecutors say, in what is believed to be the largest such case of its kind in the United States.'}]


In [0]:
print(len(bart_summary[0]["summary_text"]))

278


## T5

In [5]:
# Use T5 - PyTorch
'''
https://huggingface.co/t5-large
model = AutoModelWithLMHead.from_pretrained("t5-large")
tokenizer = AutoTokenizer.from_pretrained("t5-large")
'''

T5_summarizer = pipeline("summarization", model="t5-large", tokenizer="t5-large")

In [6]:
details2 = requests.get("https://s3.amazonaws.com/models.huggingface.co/bert/t5-3b-config.json")
t5_large_config = json.loads(details2.text)
tasks(t5_large_config, search='summarization')

Tasks:
- summarization
- translation_en_to_de
- translation_en_to_fr
- translation_en_to_ro


Detalles de summarization
{'early_stopping': True, 'length_penalty': 2.0, 'max_length': 200, 'min_length': 30, 'no_repeat_ngram_size': 3, 'num_beams': 4, 'prefix': 'summarize: '}


In [0]:
t5_summary = T5_summarizer(TEXT_TO_SUMMARIZE, min_length=56, max_length=250)
print(t5_summary)

Token indices sequence length is longer than the specified maximum sequence length for this model (527 > 512). Running this sequence through the model will result in indexing errors


In [0]:
print(len(t5_summary[0]["summary_text"]))

## Resultados

BART

Liana Barrientos has been married 10 times, sometimes within two weeks of each other. Prosecutors say the marriages were part of an immigration scam. She is believed to still be married to four men, and at one time, she was married to eight. Her eighth husband, Rashid Rajput, was deported in 2006.

BART XSUM

A New York woman has pleaded not guilty to charges that she lied on a marriage license in order to get married 10 times, including to eight different men from different countries, prosecutors say, in what is believed to be the largest such case of its kind in the United States.

T5

Liana barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree" she has been married 10 times, with nine of her marriages occurring between 1999 and 2002 . prosecutors say the marriages were part of an immigration scam .